<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# I'm gonna import with pandas - it'll be good practice :)
import pandas as pd

df = pd.read_fwf("https://www.gutenberg.org/files/100/100-0.txt").reset_index()

df = df.drop('Unnamed: 0', axis=1)
df = df.dropna().reset_index(drop=True)

df = df[63:138996].reset_index(drop=True)  # The actual text

data = df['index'].tolist()

In [0]:
import re

new_data = " ".join(data)

final_data = re.sub(r'[^a-zA-Z^0-9]', ' ', new_data)  # Filter our characters


characters = list(set(final_data))

In [0]:
# Create character-integer lookup.

char_int = {character:integer for integer, character in enumerate(characters)}
int_char = {integer:character for integer, character in enumerate(characters)}

In [4]:
# Create sequences

max_len = 40
step = 5
sequences = []
next_char = []

encoded = [char_int[c] for c in final_data]

for i in range(0, len(encoded) - max_len, step):
  sequences.append(encoded[i: i + max_len])
  next_char.append(encoded[i + max_len])
print('Sequences: ', len(sequences))

Sequences:  1055018


In [5]:
# Now, we need to create our X and y variables.
import numpy as np

X = np.zeros((len(sequences), max_len, len(characters)), dtype=np.bool)
y = np.zeros((len(sequences), len(characters)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    X[i, t, char] = 1
  y[i, next_char[i]] = 1

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (1055018, 40, 63)
y shape: (1055018, 63)


In [6]:
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

# Build our model.
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(characters))))
model.add(Dense(len(characters), activation='softmax'))

# Compile the model
model.compile(optimizer='nadam', loss='categorical_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               98304     
_________________________________________________________________
dense (Dense)                (None, 63)                8127      
Total params: 106,431
Trainable params: 106,431
Non-trainable params: 0
_________________________________________________________________


In [0]:
import random
import sys
from tensorflow.keras.callbacks import LambdaCallback

def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(final_data) - max_len - 1)
    
    generated = ''
    
    sentence = final_data[start_index: start_index + max_len]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, max_len, len(characters)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
# Fit the model

model.fit(X, y,
          batch_size=32,
          epochs=50,
          callbacks=[print_callback])  

Epoch 1/50
32964/32970 [============================>.] - ETA: 0s - loss: 1.9518
----- Generating text after Epoch: 0
----- Generating with seed: "LEAR  A plague upon you  murderers  trai"
LEAR  A plague upon you  murderers  trailing rongfel  lord  I tower Cathill now  She Madis and the wast  for we persing  a Calice of drey ate er walands for coptian  PAMONH  That my seblant bustor tho gnowed uphise with to be news and theme  The say Casine  My cull s diender a breesal  Heaben the nace ob dool herelf             PHe  sir  and that own forishers  Entel Entar woldinigh  if the mord  I ll ne  and when appasiens when too tri
32970/32970 [==============================] - 176s 5ms/step - loss: 1.9518
Epoch 2/50
32964/32970 [============================>.] - ETA: 0s - loss: 1.6585
----- Generating text after Epoch: 1
----- Generating with seed: "g foolish things  But now he throws that"
g foolish things  But now he throws that I walk before me  This punt d tempers  thour once so look  Duke 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN